In [1]:
import numpy as np
import pandas as pd
import sklearn 
import imblearn
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
### define the sensitive attribute 'Age Group' or 'sex '###
sen_att = 'Age Group'

In [3]:
german_data = pd.read_csv('german_data.csv')


print(german_data)

     stat_check_acc  duration_month  credit_history  purpose  credit_amount  \
0                 1               6               5        4           1169   
1                 2              48               3        4           5951   
2                 4              12               5        7           2096   
3                 1              42               3        3           7882   
4                 1              24               4        1           4870   
..              ...             ...             ...      ...            ...   
995               4              12               3        3           1736   
996               1              30               3        2           3857   
997               4              12               3        4            804   
998               1              45               3        4           1845   
999               2              45               5        2           4576   

     Age Group  savings_bonds  employment_since  in

In [4]:
from sklearn.model_selection import train_test_split
X = german_data.loc[:, german_data.columns != 'approval']
y = german_data.loc[:, german_data.columns == 'approval']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify = y, random_state=0)

### remove the sensitive attributes for execution but store them for metrics later ###

X_test_sen = X_test.loc[:,X_test.columns == sen_att]
X_test = X_test.loc[:, X_test.columns != sen_att]

X_train = X_train.loc[:, X_train.columns != sen_att]

### apply SMOTE to the training set ###
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0)

columns = X_train.columns
os_data_X, os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['approval'])



In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler



os_data_y_2 =os_data_y.to_numpy()
os_data_y_2 =os_data_y_2.ravel()

logit = LogisticRegression(penalty='l2', solver = 'liblinear', max_iter = 5000)
#logit = LogisticRegression()
logit.fit(os_data_X, os_data_y_2)


y_pred =  logit.predict(X_test)


from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

matrix = sklearn.metrics.confusion_matrix(y_test, y_pred)
accuracy_score = sklearn.metrics.accuracy_score(y_test, y_pred)
print(matrix, accuracy_score)


              precision    recall  f1-score   support

           0       0.57      0.71      0.63        75
           1       0.86      0.77      0.81       175

    accuracy                           0.75       250
   macro avg       0.71      0.74      0.72       250
weighted avg       0.77      0.75      0.76       250

[[ 53  22]
 [ 40 135]] 0.752


In [7]:
### identify privileged and unprivileged groups ### 
privileged_groups = [{sen_att: 0}]
unprivileged_groups = [{sen_att: 1}]


In [8]:
## #adjust  predictions for use AIF360 Library ###

fair_test_df = pd.concat([X_test, X_test_sen], axis=1, join="inner")
fair_test_df = pd.concat([fair_test_df, y_test], axis=1, join="inner")

In [9]:
# Metrics function
from collections import OrderedDict
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

dataset = StandardDataset(fair_test_df, 
                          label_name='approval', 
                          favorable_classes=[1], 
                          protected_attribute_names=[sen_att], 
                          privileged_classes=[[0]])

def fair_metrics(dataset, y_pred, disp = True):
    dataset_pred =dataset.copy()
    dataset_pred.labels = y_pred
        
    attr = dataset_pred.protected_attribute_names[0]
    
    idx = dataset_pred.protected_attribute_names.index(attr)
    privileged_groups =  [{attr:dataset_pred.privileged_protected_attributes[idx][0]}] 
    unprivileged_groups = [{attr:dataset_pred.unprivileged_protected_attributes[idx][0]}] 

    classified_metric_pred = ClassificationMetric(dataset, dataset_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

    metric_pred = BinaryLabelDatasetMetric(dataset_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

    metrics = OrderedDict()
    metrics["Balanced accuracy"] = 0.5*(classified_metric_pred.true_positive_rate()+
                                             classified_metric_pred.true_negative_rate())
    metrics["Statistical parity difference"] = classified_metric_pred.statistical_parity_difference()
    metrics["Disparate impact"] = classified_metric_pred.disparate_impact()
    metrics["Average odds difference"] = classified_metric_pred.average_odds_difference()
    metrics["Equal opportunity difference"] = classified_metric_pred.equal_opportunity_difference()
        
    if disp:
        for k in metrics:
            print("%s = %.4f" % (k, metrics[k]))


fair_metrics(dataset, y_pred)

Balanced accuracy = 0.7390
Statistical parity difference = 0.0324
Disparate impact = 1.0519
Average odds difference = -0.0470
Equal opportunity difference = 0.1130
